In [0]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np

In [0]:
data = pd.read_csv('./housing-prices/train.csv')
test = pd.read_csv('./housing-prices/test.csv')

In [115]:
data[0:3]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [0]:
cat_columns = data.select_dtypes(['object']).columns
tcat_columns = test.select_dtypes(['object']).columns

In [117]:
data.select_dtypes(exclude='object')

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000
5,6,50,85.0,14115,5,5,1993,1995,0.0,732,...,40,30,0,320,0,0,700,10,2009,143000
6,7,20,75.0,10084,8,5,2004,2005,186.0,1369,...,255,57,0,0,0,0,0,8,2007,307000
7,8,60,NaN,10382,7,6,1973,1973,240.0,859,...,235,204,228,0,0,0,350,11,2009,200000
8,9,50,51.0,6120,7,5,1931,1950,0.0,0,...,90,0,205,0,0,0,0,4,2008,129900
9,10,190,50.0,7420,5,6,1939,1950,0.0,851,...,0,4,0,0,0,0,0,1,2008,118000


In [0]:
data[cat_columns] = data[cat_columns].astype('category')
# data[bool_columns] = data[bool_columns].astype('int')

test[tcat_columns] = test[tcat_columns].astype('category')

In [0]:
data[cat_columns] = data[cat_columns].apply(lambda x: x.cat.codes)
test[tcat_columns] = test[tcat_columns].apply(lambda x: x.cat.codes)

In [0]:
# data.hist(column=10, by=None, grid=True, xlabelsize=None, xrot=None, ylabelsize=None, yrot=None, ax=None, sharex=False, sharey=False, figsize=None, layout=None, bins=10)
data.hist(figsize=(20,25))

In [0]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression

In [0]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(data)
imp.fit(test)
data.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [0]:
X = data.drop(['Id', 'SalePrice'], axis=1)
y = data['SalePrice']

test = test.drop(['Id'], axis=1)

In [0]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
model = LinearRegression()

In [141]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [142]:
model.predict(X_train)

array([137877.9862623 , 184176.78873645, 110108.77840167, ...,
       117882.19380212, 166318.612862  , 203608.80906339])

In [143]:
model.score(X_test, y_test)

0.8187017930012577

In [0]:
prediction = model.predict(test)

In [0]:
b = np.dstack(( np.arange(1461, 2920), prediction ))[0]

In [0]:
np.savetxt('housing_submission.csv', b, fmt="%d,%f", newline='\r\n', header='Id,SalePrice', comments='')

In [0]:
def rmsle(y_pred, y_test) : 
    assert len(y_test) == len(y_pred)
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_test))**2))

In [169]:
rmsle(model.predict(X_train), y_train)

0.14393865829852068